# LinkedIn Hard Mode
## Data cleaning

In [4]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [130]:
df = pd.read_csv('./data/LIHM post data.csv', parse_dates=['post_date'])

* Each day column includes a string of impressions, reactions, and comments

In [131]:
df.head()

,post_date,topic,word_count,n_images,top_company,top_title,day_1,day_2,day_3,day_4,...,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30
0,2022-01-03,beginning,396,1,US Army,software developer,"2436, 21, 27","2656, 24, 27","2733, 24, 29","2770, 24, 29",...,"2925, 24, 29","2932, 24, 29","2936, 24, 29","2940, 24, 29","2943, 24, 29","2949, 24, 29","2954, 24, 30","2957, 24, 30","2963, 24, 30","2966, 24, 30"
1,2022-01-04,intro,427,0,Amazon,software developer,"5381, 37, 28","6142, 44, 36","6533, 46, 36","6732, 46, 36",...,"7094, 46, 37","7097, 46, 37","7099, 46, 37","7103, 46, 37","7107, 46, 37","7111, 46, 37","7115, 46, 37","7121, 46, 37","7125, 46, 37","7128, 46, 37"
2,2022-01-05,culvers dream,560,0,"VJS Construction Services, Inc.",software developer,"1231, 13, 13","1297, 14, 15","1315, 14, 15","1353, 15, 15",...,"1488, 15, 16","1491, 15, 16","1495, 15, 16","1499, 15, 16","1506, 15, 16","1509, 15, 16","1515, 15, 16","1518, 15, 16",NaN,"1525, 15, 16"
3,2022-01-06,practicing,491,1,Christopher Newport University,software developer,"646, 14, 8","771, 18, 11","859, 18, 12","930, 20, 12",...,"1085, 21, 12","1088, 21, 12","1094, 21, 12","1101, 21, 12","1103, 21, 12","1108, 21, 12","1113, 21, 12",NaN,"1120, 21, 12","1126, 21, 12"
4,2022-01-07,goal-setting 1,465,0,Christopher Newport University,data analyst,"859, 10, 16","922, 10, 16","948, 10, 16","975, 11, 18",...,"1124, 11, 23","1130, 11, 23","1135, 11, 23","1138, 11, 23","1142, 11, 23","1147, 11, 23",NaN,"1152, 11, 23","1155, 11, 23","1158, 11, 23"


### First, deal with nulls
* Since this data has a temporal component, whenever possible we need to impute the last known truth value.

In [134]:
df.loc[:, 'day_1':'day_30'] = df.loc[:, 'day_1':'day_30'].fillna(method='pad', axis=1)

* After doing that, we are left with two nulls, days where I missed the first day of recording data.
* Since it's a small dataset, I'll impute manually, by using the trend of a similarly-performing post to estimate what day_1 would have been.

In [135]:
df[df['day_1'].isna()]

,post_date,topic,word_count,n_images,top_company,top_title,day_1,day_2,day_3,day_4,...,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30
9,2022-01-12,contentment,516,0,Prudential Financial,software developer,NaN,"602, 8, 9","602, 8, 9","731, 9, 9",...,"1078, 11, 9","1078, 11, 9","1082, 11, 9","1086, 11, 9","1089, 11, 9","1091, 11, 9","1096, 11, 9","1099, 11, 9","1102, 11, 9","1102, 11, 9"
11,2022-01-14,debugging,255,0,Ramsey Solutions,software developer,NaN,"582, 7, 3","602, 7, 3","650, 7, 3",...,"806, 7, 6","810, 7, 6","813, 7, 6","816, 7, 6","821, 7, 6","827, 7, 6","830, 7, 6","830, 7, 6","837, 7, 6","840, 7, 6"


* For the post on 01.12, I will use as a comparison the post from 01.09, which had 639, 7, 10 on day_2 and 552, 6, 7 on day_1
* For the post on 01.14, I will use as a comparison the post from 01.20, which had 554, 8, 11 on day_2 and 490, 8, 9 on day_1

In [136]:
to_impute_0112 = f'{int(602 * (552/639))}, {int(8 * (6/7))}, {int(9 * (7/10))}'
to_impute_0114 = f'{int(582 * (490/554))}, {int(7 * (8/8))}, {int(3 * (9/11))}'

In [137]:
df.loc[9, 'day_1'] = to_impute_0112
df.loc[11, 'day_1'] = to_impute_0114

### Splitting data into more columns!

* Each of the cells with engagement data is a string with 3 values, so they need to be split and turned to ints

In [138]:
%%time
for i in range(1, 31):
    col_name = f'day_{i}'
    df[col_name + '_imp'] = df[col_name].apply(lambda x: int(x.split(', ')[0]))
    df[col_name + '_reax'] = df[col_name].apply(lambda x: int(x.split(', ')[1]))
    df[col_name + '_comm'] = df[col_name].apply(lambda x: int(x.split(', ')[2]))
    df = df.drop(col_name, axis=1)

Wall time: 55.5 ms


### Save the cleaned dataset as a CSV for Tableau

In [142]:
df.to_csv('./data/cleaned_dataset.csv', index=False)